# Benchmark

In [ ]:
using BenchmarkTools

In [ ]:
include("src/serialmd.jl")

In [ ]:
include("src/utilmd.jl")

In [ ]:
# Valori originali del test di Crabb
const box_size = 1e3;
const dim = 3;
const dt = 0.01;
const num_part_types = 2;
const nsteps = 1000;
const periodic = true;
const sinterval = 10;

### Serial version benchmark

In [ ]:
function benchmark_serial(part_num)
    # Genera stato iniziale del sistema sulla base dei valori originali
    pos, vel, acc, masses, interactions, ptypes = Utilmd.random_data(dim, part_num, num_part_types, box_size)
    # Benchmark
    @benchmark Serialmd.dynamics_sim!($nsteps, $sinterval, false, $dt, $pos, $vel, $acc, $masses, $interactions, $ptypes, $box_size, $periodic, true)
end

In [ ]:
# Serial test, 100 particelle
benchmark_serial(100)

In [ ]:
# Serial test, 100 particelle
benchmark(Serialmd.dynamics_sim!, 500)

In [ ]:
# Serial test, 100 particelle
benchmark(Serialmd.dynamics_sim!, 1000)

### CPU Multi Thread Version Benchmark

In [ ]:
# TODO Con/Senza Hyperthreading

function benchmark_CPUMT(part_num)
    # Genera stato iniziale del sistema sulla base dei valori originali
    pos, vel, acc, masses, interactions, ptypes = Utilmd.random_data(dim, part_num, num_part_types, box_size)
    # Benchmark
    @benchmark CPUMTmd.dynamics_sim!($nsteps, $sinterval, false, $dt, $pos, $vel, $acc, $masses, $interactions, $ptypes, $box_size, $periodic, true)
end

In [ ]:
# CPU-MT test 100 particelle
benchmark_CPUMT(100)

In [ ]:
# CPU-MT test 500 particelle
benchmark_CPUMT(500)

In [ ]:
# CPU-MT test 1000 particelle
benchmark_CPUMT(1000)

### CUDA Version Benchmark

In [ ]:
function benchmark_CUDA(part_num)
    # Genera stato iniziale del sistema sulla base dei valori originali
    pos, vel, acc, masses, interactions, ptypes = Utilmd.random_data(dim, part_num, num_part_types, box_size)
    # Benchmark
    @benchmark CUDAmd.dynamics_sim!($nsteps, $sinterval, false, $dt, $pos, $vel, $acc, $masses, $interactions, $ptypes, $box_size, $periodic, true)
end

In [ ]:
# CUDA test 100 particelle
benchmark_CUDA(100)

In [ ]:
# CUDA test 500 particelle
benchmark_CUDA(500)

In [ ]:
# CUDA test 1000 particelle
benchmark_CUDA(1000)